In [1]:
import sys
sys.path.append('../')
from lib import mesh_sampling
import numpy as np
import json
import os
import copy
from facemesh import FaceData
import time
import pickle
import trimesh

try:
    import psbody.mesh
    found = True
except ImportError:
    found = False
if found:
    from psbody.mesh import Mesh, MeshViewer, MeshViewers

from autoencoder_dataset import autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoder, SpiralAutoencoder_extra_conv

from test_funcs import test_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'trimesh'
root_dir = '/data/gb318/datasets/'

name = 'sliced'
dataset = 'COMA'    

GPU = True
device_idx = 9
torch.cuda.get_device_name(device_idx)

'GeForce RTX 2080 Ti'

In [2]:
args = {}

generative_model = 'autoencoder'
dilation_flag = False
hardcode_down_ref = False
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'
downsample_config = ''

if dataset == 'COMA':
    reference_mesh_file = os.path.join(root_dir, dataset,'preprocessed/templates/template.obj')
    downsample_directory = os.path.join(root_dir, dataset,'preprocessed/templates',downsample_method,downsample_config)
    ds_factors = [4, 4, 4, 4]
    step_sizes = [1, 1, 1, 1, 1]
    filter_sizes_enc = [[3, 16, 16, 16, 32],[[],[],[],[],[]]]
    filter_sizes_dec = [[32, 32, 16, 16, 3],[[],[],[],[],[]]]
    if dilation_flag:
        dilation=[2, 2, 2, 1, 1] 
    else:
        dilation = None

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/higher_order_'+ generative_model,\
                                        downsample_method, downsample_config,'injection_1st_order_reimpl'),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size':16, 'num_epochs':300, 'eval_frequency':200, 'num_workers': 4,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz':16, 
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,
        'injection': False, 'residual': False, 
        
        'lr':1e-3, 
        'regularization': 5e-5,         
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': False,
        
        'mode':'test', 'shuffle': True, 'nVal': 100, 'normalization': True}

if generative_model == 'autoencoder':
    args['results_folder'] = os.path.join(args['results_folder'],\
                                          'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

if hardcode_down_ref:
    if dataset == 'COMA' and downsample_method == 'COMA_downsample':
        reference_points = [[3567,4051,4597],
                            [1010,1081,1170],
                            [256, 276, 295],
                            [11, 69, 74],
                            [17, 17, 17]]
    elif dataset == 'COMA' and downsample_method == 'meshlab_downsample' and downsample_config == 'preserve_topology=True_preserve_boundary=False':
        reference_points = [[3567, 4051, 4597],
                             [1105, 1214, 1241],
                             [289, 310, 318],
                             [70, 80, 85],
                             [2, 19, 24]]
    else:
        raise NotImplementedError
else:
    if dataset == 'COMA':
        reference_points = [[3567,4051,4597]]
        

In [3]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    facedata = FaceData(nVal=args['nVal'], train_file=args['data']+'/train.npy',
                             test_file=args['data']+'/test.npy', reference_mesh_file=args['reference_mesh_file'],
                             pca_n_comp=args['nz'], normalization = args['normalization'],\
                             meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', facedata.mean)
    np.save(args['data']+'/std.npy', facedata.std)
else:
    facedata = FaceData(nVal=args['nVal'], train_file=args['data']+'/train.npy',\
                        test_file=args['data']+'/test.npy', reference_mesh_file=args['reference_mesh_file'],\
                        pca_n_comp=args['nz'], normalization = args['normalization'],\
                        meshpackage = meshpackage, load_flag = False)
    facedata.mean = np.load(args['data']+'/mean.npy')
    facedata.std = np.load(args['data']+'/std.npy')
    facedata.n_vertex = facedata.mean.shape[0]
    facedata.n_features = facedata.mean.shape[1]

if not os.path.exists(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl')):
    if facedata.meshpackage == 'trimesh':
        raise NotImplementedError
    print("Generating Transform Matrices ..")


    if downsample_method == 'COMA_downsample':
        M,A,D,U,F = mesh_sampling.generate_transform_matrices(facedata.reference_mesh, args['ds_factors'])
    elif downsample_method == 'meshlab_downsample':
        M,A,D,U,F = mesh_sampling.generate_transform_matrices_given_downsamples(facedata.reference_mesh,                                                                                args['downsample_directory'],                                                                                len(args['ds_factors']))
    else:
        raise NotImplementedError(downsample_method)
        
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'wb') as fp:
        M_verts_faces = [(M[i].v, M[i].f) for i in range(len(M))]
        pickle.dump({'M_verts_faces':M_verts_faces,'A':A,'D':D,'U':U,'F':F}, fp)
else:
    print("Loading Transform Matrices ..")
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'rb') as fp:
        downsampling_matrices = pickle.load(fp,encoding = 'latin1')
            
    M_verts_faces = downsampling_matrices['M_verts_faces']
    if facedata.meshpackage == 'mpi-mesh':
        M = [Mesh(v=M_verts_faces[i][0], f=M_verts_faces[i][1]) for i in range(len(M_verts_faces))]
    elif facedata.meshpackage == 'trimesh':
        M = [trimesh.base.Trimesh(vertices=M_verts_faces[i][0], faces=M_verts_faces[i][1], process = False)             for i in range(len(M_verts_faces))]
    A = downsampling_matrices['A']
    D = downsampling_matrices['D']
    U = downsampling_matrices['U']
    F = downsampling_matrices['F']
        

if not hardcode_down_ref:
    print("Calculating reference points for downsampled versions..")
    for i in range(len(args['ds_factors'])):
        if facedata.meshpackage == 'mpi-mesh':
            dist = euclidean_distances(M[i+1].v, M[0].v[reference_points[0]])
        elif facedata.meshpackage == 'trimesh':
            dist = euclidean_distances(M[i+1].vertices, M[0].vertices[reference_points[0]])
        reference_points.append(np.argmin(dist,axis=0).tolist())



Loading data .. 
Loading Transform Matrices ..
Calculating reference points for downsampled versions..


In [4]:
if facedata.meshpackage == 'mpi-mesh':
    sizes = [x.v.shape[0] for x in M]
elif facedata.meshpackage == 'trimesh':
    sizes = [x.vertices.shape[0] for x in M]
Adj, Trigs = get_adj_trigs(A, F, facedata.reference_mesh, meshpackage = facedata.meshpackage)

spirals_np, spiral_sizes,spirals = generate_spirals(args['step_sizes'], M, Adj, Trigs, \
                                                    reference_points = reference_points, \
                                                    dilation = args['dilation'], random = False, \
                                                    meshpackage = facedata.meshpackage, counter_clockwise = True)

bU = []
bD = []
for i in range(len(D)):
    d = np.zeros((1,D[i].shape[0]+1,D[i].shape[1]+1))
    u = np.zeros((1,U[i].shape[0]+1,U[i].shape[1]+1))
    d[0,:-1,:-1] = D[i].todense()
    u[0,:-1,:-1] = U[i].todense()
    d[0,-1,-1] = 1
    u[0,-1,-1] = 1
    bD.append(d)
    bU.append(u)


spiral generation for hierarchy 0 (5023 vertices) finished
spiral generation for hierarchy 1 (1256 vertices) finished
spiral generation for hierarchy 2 (314 vertices) finished
spiral generation for hierarchy 3 (79 vertices) finished
spiral generation for hierarchy 4 (20 vertices) finished
spiral sizes for hierarchy 0:  9
spiral sizes for hierarchy 1:  9
spiral sizes for hierarchy 2:  9
spiral sizes for hierarchy 3:  9
spiral sizes for hierarchy 4:  8


In [5]:
# pytorch stuff

torch.manual_seed(args['seed'])

if GPU:
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

tspirals = [torch.from_numpy(s).long().to(device) for s in spirals_np]
tD = [torch.from_numpy(s).float().to(device) for s in bD]
tU = [torch.from_numpy(s).float().to(device) for s in bU]

cuda:9


In [87]:
# Building model, optimizer, and loss function


dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          facedata = facedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=16,\
                                     shuffle = False, num_workers = args['num_workers'])



if 'autoencoder' in args['generative_model']:
        model = SpiralAutoencoder_extra_conv(filters_enc = args['filter_sizes_enc'],   
                                             filters_dec = args['filter_sizes_dec'],
                                             latent_size=args['nz'],
                                             sizes=sizes,
                                             spiral_sizes=spiral_sizes,
                                             spirals=tspirals,
                                             D=tD, U=tU,device=device,
                                             injection = args['injection'],
                                             residual = args['residual']).to(device)



In [88]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)


Total number of parameters is: 48547
SpiralAutoencoder_extra_conv(
  (conv): ModuleList(
    (0): SpiralConv(
      (conv): Linear(in_features=27, out_features=16, bias=False)
      (activation): ELU(alpha=1.0)
    )
    (1): SpiralConv(
      (conv): Linear(in_features=144, out_features=16, bias=False)
      (activation): ELU(alpha=1.0)
    )
    (2): SpiralConv(
      (conv): Linear(in_features=144, out_features=16, bias=False)
      (activation): ELU(alpha=1.0)
    )
    (3): SpiralConv(
      (conv): Linear(in_features=144, out_features=32, bias=False)
      (activation): ELU(alpha=1.0)
    )
  )
  (fc_latent_enc): Linear(in_features=672, out_features=16, bias=True)
  (fc_latent_dec): Linear(in_features=16, out_features=672, bias=True)
  (dconv): ModuleList(
    (0): SpiralConv(
      (conv): Linear(in_features=288, out_features=32, bias=False)
      (activation): ELU(alpha=1.0)
    )
    (1): SpiralConv(
      (conv): Linear(in_features=288, out_features=16, bias=False)
      (act

In [89]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     facedata, mm_constant = 1000)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

  0%|          | 0/129 [00:00<?, ?it/s]

loading checkpoint from file /data/gb318/datasets/COMA/results/higher_order_autoencoder/COMA_downsample/injection_1st_order_reimpl/latent_16/checkpoints/sliced/checkpoint.pth.tar


100%|██████████| 129/129 [00:03<00:00, 37.79it/s]


autoencoder: normalized loss 0.11883243918418884
autoencoder: euclidean distance in mm= 0.6359777450561523


In [97]:
with torch.no_grad():
    mm_constant = 1000
    shapedata_mean = torch.Tensor(facedata.mean).to(device)
    shapedata_std = torch.Tensor(facedata.std).to(device)
    x = dataset_test.__getitem__(500)['points'].to(device).unsqueeze(0)
    x_recon = model(x)  
    if dataset_test.dummy_node:
        x_recon = x_recon[:,:-1]
        x = x[:,:-1]
    else:
        pass
    x_recon = ((x_recon * shapedata_std + shapedata_mean) * mm_constant).cpu().numpy()
    x = ((x * shapedata_std + shapedata_mean) * mm_constant).cpu().numpy()
    error_norm = np.sqrt(np.sum((x_recon - x)**2,axis=2))
    template = trimesh.load(args['reference_mesh_file'], process = False)
    template.vertices = x.squeeze()
#     clr = trimesh.visual.color.interpolate(error_norm,color_map='jet')
#     template.visual = trimesh.visual.create_visual(vertex_colors = clr)
#     template.show()
    trimesh.exchange.export.export_mesh(template, os.path.join('/homes/gb318/Downloads/temp_texture.ply'));



In [ ]:
3/(195.73+216.55+213.72) # simple linear

In [ ]:
3/(187.59+207.07+207.52) # simple

In [ ]:
3/(175 + 193.38 + 197.76) # full linear

In [ ]:
3/(179.23 + 196.16 + 207.55) # full

In [ ]:
3/(167.01+173.61+180.88)

In [ ]:
3/(178.59+194.72+200.05)

In [ ]:
3/(176.45+178.29+176.06)

In [ ]:
5/(81.94+83.07+83.52+82.79+84.39)